Eligibility

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice


In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [4]:
curs.execute("select * from PG_TABLES where schemaname = 'mimiciv_derived';")
rows = curs.fetchall( ) #결과가지고옴

for row in rows:
    print(row)

('mimiciv_derived', 'age', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'icustay_detail', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'icustay_hourly', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'bg', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'blood_differential', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'cardiac_marker', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'chemistry', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'coagulation', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'complete_blood_count', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'creatinine_baseline', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'enzyme', 'mai_admin', None, False, False, False, False)
('mimiciv_derived', 'gcs', 'mai_admin', None, False, False, False, False)
('mimiciv_deriv

In [5]:
curs.execute("select * from mimiciv_derived.age;")
columns_name = [desc[0] for desc in curs.description]
age = pd.DataFrame(curs.fetchall(), columns=columns_name) 
age #더 디테일한 나이 나옴

,subject_id,hadm_id,admittime,anchor_age,anchor_year,age
0,10000084,23052089,2160-11-21 01:56:00,72,2160,72.89004154931676958169
1,10000084,29888819,2160-12-28 05:11:00,72,2160,72.99171500600210879971
2,10000764,27897940,2132-10-14 23:31:00,86,2132,86.78846315897709220492
3,10000886,21927847,2178-05-08 08:07:00,18,2178,18.34864061209949689369
4,10000980,20897796,2193-08-15 01:01:00,73,2186,80.6197216122765485
...,...,...,...,...,...,...
431226,19999784,29355057,2119-10-17 10:28:00,57,2119,57.79245024151414982699
431227,19999784,29889147,2120-10-25 09:43:00,57,2119,58.8163433041958787
431228,19999784,29956342,2121-01-31 00:00:00,57,2119,59.0835500846014423
431229,19999828,25744818,2149-01-08 16:44:00,46,2147,48.0224870694559285


In [6]:
curs.execute("select * from mimiciv_derived.icustay_detail;")
columns_name = [desc[0] for desc in curs.description]
icustay_detail = pd.DataFrame(curs.fetchall(), columns=columns_name) 
icustay_detail 

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,hospital_expire_flag,hospstay_seq,first_hosp_stay,icu_intime,icu_outtime,los_icu,icustay_seq,first_icu_stay
0,10000032,29079034,39553978,F,2180-09-09,2180-07-23 12:35:00,2180-07-25 17:55:00,2.2222222222222222,52.55996929585194352116,WHITE,0,1,True,2180-07-23 14:00:00,2180-07-23 23:50:47,0.41,1,True
1,10000980,26913865,39765666,F,2193-08-26,2189-06-27 07:38:00,2189-07-03 03:00:00,5.8069444444444444,76.4862311989189511,BLACK/AFRICAN AMERICAN,0,1,True,2189-06-27 08:42:00,2189-06-27 20:38:27,0.50,1,True
2,10001217,24597018,37067082,F,None,2157-11-18 22:56:00,2157-11-25 18:00:00,6.7944444444444444,55.88148557820720386909,WHITE,0,1,True,2157-11-20 19:18:02,2157-11-21 22:08:00,1.12,1,True
3,10001217,27703517,34592300,F,None,2157-12-18 16:58:00,2157-12-24 14:55:00,5.9145833333333333,55.96294222582409592666,WHITE,0,2,False,2157-12-19 15:42:24,2157-12-20 14:27:41,0.95,1,True
4,10001725,25563031,31205490,F,None,2110-04-11 15:08:00,2110-04-14 15:00:00,2.9944444444444444,46.27551748034332184019,WHITE,0,1,True,2110-04-11 15:52:22,2110-04-12 23:59:56,1.34,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,M,None,2148-11-19 10:00:00,2148-12-04 16:25:00,15.2673611111111111,43.8841608212272046,WHITE,0,1,True,2148-11-19 14:23:43,2148-11-26 13:12:15,6.95,1,True
73177,19999625,25304202,31070865,M,None,2139-10-10 18:06:00,2139-10-16 03:30:00,5.3916666666666667,82.7734930995522603,WHITE,0,1,True,2139-10-10 19:18:00,2139-10-11 18:21:28,0.96,1,True
73178,19999828,25744818,36075953,F,None,2149-01-08 16:44:00,2149-01-18 17:00:00,10.0111111111111111,48.0224870694559285,WHITE,0,1,True,2149-01-08 18:12:00,2149-01-10 13:11:02,1.79,1,True
73179,19999840,21033226,38978960,M,2164-09-17,2164-09-10 13:47:00,2164-09-17 13:42:00,6.9965277777777778,58.69426381838768694607,WHITE,1,1,True,2164-09-12 09:26:28,2164-09-17 16:35:15,5.30,1,True


In [7]:
curs.execute("select * from mimiciv_derived.icustay_hourly;")
columns_name = [desc[0] for desc in curs.description]
icustay_hourly = pd.DataFrame(curs.fetchall(), columns=columns_name) 
icustay_hourly 

,stay_id,hr,endtime
0,39553978,-24,2180-07-22 15:00:00
1,39553978,-23,2180-07-22 16:00:00
2,39553978,-22,2180-07-22 17:00:00
3,39553978,-21,2180-07-22 18:00:00
4,39553978,-20,2180-07-22 19:00:00
...,...,...,...
7802415,36195440,42,2145-11-04 18:00:00
7802416,36195440,43,2145-11-04 19:00:00
7802417,36195440,44,2145-11-04 20:00:00
7802418,36195440,45,2145-11-04 21:00:00


In [10]:
curs.execute("select * from mimiciv_derived.cardiac_marker;")
columns_name = [desc[0] for desc in curs.description]
cardiac_marker = pd.DataFrame(curs.fetchall(), columns=columns_name) 
cardiac_marker 

,subject_id,hadm_id,charttime,specimen_id,troponin_t,ck_mb,ntprobnp
0,10120037,29159481.0,2161-03-04 19:00:00,42,___,1.0,NaN
1,10155734,NaN,2133-12-23 18:22:00,509,None,NaN,4531.0
2,11948014,26716559.0,2163-08-28 16:50:00,878,___,4.0,NaN
3,11050359,NaN,2129-07-03 10:35:00,1954,None,10.0,NaN
4,18277986,24920171.0,2154-03-30 21:35:00,2656,___,NaN,NaN
...,...,...,...,...,...,...,...
295241,12064210,22982565.0,2137-01-17 06:40:00,99998872,None,1.0,NaN
295242,16739274,NaN,2184-12-17 09:25:00,99998919,None,NaN,436.0
295243,12882274,25494221.0,2119-04-16 22:10:00,99999683,None,NaN,753.0
295244,14723565,26017332.0,2167-01-30 11:44:00,99999742,___,NaN,NaN


In [11]:
curs.execute("select * from mimiciv_derived.creatinine_baseline;")
columns_name = [desc[0] for desc in curs.description]
creatinine_baseline = pd.DataFrame(curs.fetchall(), columns=columns_name) 
creatinine_baseline 

,hadm_id,gender,age,scr_min,ckd,mdrd_est,scr_baseline
0,25911675,F,78.3899823039701531,2.1,1,0.78757470658570392397,2.100000
1,26913865,F,76.4862311989189511,1.7,1,0.79098819761518223020,1.700000
2,29654838,F,75.0061679805596168,1.4,1,0.79371177734565948239,1.400000
3,24016413,F,46.80303492844013923189,NaN,0,0.86236914040957517121,0.862369
4,21544441,F,89.42163825627939831673,0.6,0,0.76954296373013937437,0.600000
...,...,...,...,...,...,...,...
431226,24935234,M,57.52003382536631724841,0.7,0,1.07706764078916628983,0.700000
431227,25715748,M,57.60913951115516105304,0.6,0,1.07677440012561015431,0.600000
431228,29889147,M,58.8163433041958787,0.8,0,1.07285336151961908447,0.800000
431229,29956342,M,59.0835500846014423,0.9,0,1.07199825097235257187,0.900000


In [12]:
curs.execute("select * from mimiciv_derived.gcs;")
columns_name = [desc[0] for desc in curs.description]
gcs = pd.DataFrame(curs.fetchall(), columns=columns_name) 
gcs 

,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


In [13]:
curs.execute("select * from mimiciv_derived.height;")
columns_name = [desc[0] for desc in curs.description]
height = pd.DataFrame(curs.fetchall(), columns=columns_name) 
height 
#icd version 이 9 가 있고 10이 있는데, 나는 그건 모르겠고, 일단 delirium 이라는 거를 찾고 싶어요

,subject_id,stay_id,charttime,height
0,10000032,39553978,2180-07-23 12:36:00,152.00
1,10001725,31205490,2110-04-11 15:52:00,157.00
2,10001884,37510196,2131-01-11 04:20:00,157.00
3,10002013,39060235,2160-05-18 10:00:00,157.00
4,10002428,33987268,2156-04-12 16:24:00,150.00
...,...,...,...,...
33469,19999287,37692584,2197-07-26 03:31:00,163.00
33470,19999287,35165301,2197-08-03 20:58:00,165.00
33471,19999297,37364566,2162-08-16 05:48:00,165.00
33472,19999442,32336619,2148-11-19 14:23:00,193.00


In [14]:
curs.execute("select * from mimiciv_derived.icp;")
columns_name = [desc[0] for desc in curs.description]
icp = pd.DataFrame(curs.fetchall(), columns=columns_name) 
icp 

,subject_id,stay_id,charttime,icp
0,10025463,38275267,2137-10-09 02:54:00,36.0
1,10025463,38275267,2137-10-09 03:15:00,11.0
2,10025463,38275267,2137-10-09 04:00:00,11.0
3,10025463,38275267,2137-10-09 05:00:00,12.0
4,10025463,38275267,2137-10-09 05:45:00,10.0
...,...,...,...,...
173268,19992885,35180845,2152-07-24 08:37:00,22.0
173269,19992885,35180845,2152-07-24 09:00:00,38.0
173270,19992885,35180845,2152-07-24 10:00:00,13.0
173271,19992885,35180845,2152-07-24 11:00:00,8.0


In [15]:
curs.execute("select * from mimiciv_derived.oxygen_delivery;")
columns_name = [desc[0] for desc in curs.description]
oxygen_delivery = pd.DataFrame(curs.fetchall(), columns=columns_name) 
oxygen_delivery 


,subject_id,stay_id,charttime,o2_flow,o2_flow_additional,o2_delivery_device_1,o2_delivery_device_2,o2_delivery_device_3,o2_delivery_device_4
0,10000032,39553978,2180-07-23 14:20:00,2.0,NaN,Nasal cannula,None,None,None
1,10000032,39553978,2180-07-23 18:00:00,2.0,NaN,Nasal cannula,None,None,None
2,10000032,39553978,2180-07-23 20:00:00,2.0,NaN,Nasal cannula,None,None,None
3,10000980,39765666,2189-06-27 09:00:00,NaN,40.0,None,None,None,None
4,10000980,39765666,2189-06-27 10:00:00,3.0,NaN,Nasal cannula,None,None,None
...,...,...,...,...,...,...,...,...,...
601541,19999625,31070865,2139-10-11 12:00:00,NaN,2.0,Nasal cannula,None,None,None
601542,19999625,31070865,2139-10-11 16:00:00,NaN,2.0,Nasal cannula,None,None,None
601543,19999987,36195440,2145-11-03 20:54:00,15.0,NaN,Face tent,None,None,None
601544,19999987,36195440,2145-11-04 10:00:00,4.0,NaN,Nasal cannula,None,None,None


In [16]:
curs.execute("select * from mimiciv_derived.ventilator_setting;")
columns_name = [desc[0] for desc in curs.description]
ventilator_setting = pd.DataFrame(curs.fetchall(), columns=columns_name) 
ventilator_setting 

,subject_id,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type
0,10000980,39765666,2189-06-27 09:00:00,NaN,27.0,27.0,11.2,NaN,393.0,528.0,NaN,5.0,NaN,NaN,CPAP/PSV,None,Drager
1,10000980,39765666,2189-06-27 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,Standby,None,None
2,10001884,37510196,2131-01-11 04:00:00,20.0,20.0,0.0,9.0,450.0,479.0,NaN,24.0,5.0,100.0,39.9,None,APV (cmv),Hamilton
3,10001884,37510196,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,None,APV (cmv),Hamilton
4,10001884,37510196,2131-01-11 11:00:00,20.0,20.0,0.0,8.3,400.0,430.0,NaN,23.0,5.2,50.0,32.3,None,APV (cmv),Hamilton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006122,19999987,36195440,2145-11-03 22:37:00,NaN,29.0,29.0,8.8,NaN,429.0,641.0,NaN,10.0,50.0,NaN,CPAP/PSV,None,None
1006123,19999987,36195440,2145-11-04 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standby,None,None
1006124,19999987,36195440,2145-11-04 07:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,None,None,None
1006125,19999987,36195440,2145-11-04 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,None,None,None


In [17]:
curs.execute("select * from mimiciv_derived.apsiii;")
columns_name = [desc[0] for desc in curs.description]
apsiii = pd.DataFrame(curs.fetchall(), columns=columns_name) 
apsiii 

,subject_id,hadm_id,stay_id,apsiii,apsiii_prob,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,12466550,23998182,30000153,44,0.08654575921862385844,7.0,6.0,0.0,0.0,NaN,...,0.0,0.0,5.0,7.0,0.0,NaN,NaN,0.0,1.0,15.0
1,13180007,27543152,30000213,44,0.08654575921862385844,0.0,7.0,0.0,6.0,NaN,...,0.0,7.0,0.0,11.0,2.0,0.0,0.0,5.0,NaN,3.0
2,18421337,22413411,30000484,79,0.33126704000552879910,1.0,15.0,2.0,8.0,NaN,...,5.0,0.0,5.0,11.0,0.0,0.0,0.0,0.0,NaN,29.0
3,12207593,22795209,30000646,29,0.04455469432701904035,1.0,15.0,0.0,9.0,NaN,...,0.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0
4,12980335,23552849,30001148,82,0.36339013544336734819,0.0,7.0,2.0,17.0,0.0,...,0.0,0.0,4.0,0.0,0.0,NaN,NaN,0.0,1.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16180713,20178444,39999301,46,0.09431596692709329452,0.0,10.0,2.0,8.0,0.0,...,0.0,4.0,7.0,7.0,0.0,0.0,0.0,5.0,0.0,0.0
73177,15498623,27161460,39999384,50,0.11174890826305648295,0.0,15.0,0.0,8.0,NaN,...,0.0,7.0,5.0,12.0,0.0,NaN,NaN,0.0,NaN,0.0
73178,11256534,26910235,39999552,81,0.35252925485606272195,7.0,15.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,2.0,48.0
73179,15403458,25335698,39999562,14,0.02243681045149745325,5.0,4.0,0.0,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [18]:
curs.execute("select * from mimiciv_derived.oasis;")
columns_name = [desc[0] for desc in curs.description]
oasis = pd.DataFrame(curs.fetchall(), columns=columns_name) 
oasis 

,subject_id,hadm_id,stay_id,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,...,resprate,resp_rate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,15726063,23069026,36415879,27,0.06110557118878299662,53.3839872681065644,6,73.0000000000000000,3,15.0,...,31.0,6.0,38.56,2.0,4580.0,0.0,0,0,0,6
1,14304873,20156152,35621403,28,0.06884295828308623481,75.1505757369999434,6,9241.9666666666666667,1,14.0,...,32.0,6.0,35.40,4.0,2390.0,1.0,0,0,0,6
2,12117555,25128945,37212487,34,0.13709860673818602936,82.37789823064038515712,9,14055.783333333333,1,9.0,...,27.0,1.0,36.6150000000000000,0.0,85.0,10.0,0,0,0,6
3,16645325,26608451,37532286,33,0.12270071663757324864,77.62581604951116124530,9,1691.9833333333333333,1,15.0,...,24.0,1.0,38.00,2.0,1015.0,5.0,0,0,0,6
4,10989579,26999802,38267080,26,0.05418719720382744723,25.87166522470033566786,3,95.0000000000000000,3,15.0,...,34.0,6.0,36.39,2.0,1225.0,5.0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,13282748,21364602,34149913,25,0.04801206627528390922,67.6392433722785928,6,58.0000000000000000,3,9.0,...,27.0,1.0,37.56,2.0,3170.0,0.0,0,0,0,6
73177,11079785,21641322,38019496,24,0.04250902048594646910,46.8008161559854684,3,10803.116666666667,1,15.0,...,43.0,6.0,39.33,2.0,3870.0,0.0,0,0,0,6
73178,10249699,24955103,34378808,24,0.04250902048594646910,52.19953475290963860186,3,23428.933333333333,2,15.0,...,41.0,6.0,35.61,4.0,NaN,NaN,0,0,0,6
73179,17747104,20403718,32048592,21,0.02939485724741537912,29.7838344926864319,3,44.0000000000000000,3,14.0,...,10.0,1.0,37.33,2.0,2400.0,1.0,0,0,0,6


In [19]:
curs.execute("select * from mimiciv_derived.sapsii;")
columns_name = [desc[0] for desc in curs.description]
sapsii = pd.DataFrame(curs.fetchall(), columns=columns_name) 
sapsii 

,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,14046553,20000094,35605481,2150-03-02 15:19:31,2150-03-03 15:19:31,98,0.982729,18,11.0,13.0,...,11.0,10.0,0.0,0.0,0.0,3.0,0.0,26.0,0,6
1,14990224,20000147,30503572,2121-08-30 16:33:54,2121-08-31 16:33:54,35,0.166523,15,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,6
2,17913090,20000351,30593599,2145-06-13 20:10:27,2145-06-14 20:10:27,24,0.058153,7,0.0,0.0,...,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,6
3,16788749,20000808,30610654,2180-01-17 19:32:10,2180-01-18 19:32:10,17,0.025876,7,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
4,16788749,20000808,35191063,2180-01-12 22:07:00,2180-01-13 22:07:00,26,0.071716,7,0.0,0.0,...,11.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,14641622,29999186,32199257,2135-07-12 01:51:00,2135-07-13 01:51:00,30,0.106398,7,0.0,5.0,...,0.0,0.0,3.0,0.0,0.0,0.0,4.0,0.0,0,8
73177,10053207,29999444,31112026,2199-12-14 19:52:26,2199-12-15 19:52:26,26,0.071716,7,4.0,0.0,...,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0,6
73178,13478841,29999498,33874605,2188-05-13 12:03:00,2188-05-14 12:03:00,34,0.152870,16,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,9,6
73179,10355856,29999625,36975675,2157-11-07 11:51:00,2157-11-08 11:51:00,32,0.128048,12,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0,8


In [20]:
curs.execute("select * from mimiciv_derived.sirs;")
columns_name = [desc[0] for desc in curs.description]
sirs = pd.DataFrame(curs.fetchall(), columns=columns_name) 
sirs 

,subject_id,hadm_id,stay_id,sirs,temp_score,heart_rate_score,resp_score,wbc_score
0,13180007,27543152,30000213,2,0.0,1.0,1.0,0.0
1,16165135,24791729,30003125,2,0.0,1.0,1.0,0.0
2,19462678,27280135,30003202,4,1.0,1.0,1.0,1.0
3,11101925,23920532,30004018,3,0.0,1.0,1.0,1.0
4,17220323,25700666,30004242,1,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
73176,16362502,27625929,39982460,2,NaN,1.0,1.0,0.0
73177,15119590,24923917,39982700,2,0.0,1.0,0.0,1.0
73178,15530538,21057134,39986775,1,0.0,0.0,1.0,0.0
73179,15368898,27299174,39990887,2,0.0,0.0,1.0,1.0


In [21]:
curs.execute("select * from mimiciv_derived.sofa;")
columns_name = [desc[0] for desc in curs.description]
sofa = pd.DataFrame(curs.fetchall(), columns=columns_name) 
sofa 

,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30000153,0,2174-09-29 12:00:00,2174-09-29 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,0,0,0,0,0,0
1,30000153,1,2174-09-29 13:00:00,2174-09-29 14:00:00,NaN,442.0,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,0,0,0
2,30000153,2,2174-09-29 14:00:00,2174-09-29 15:00:00,NaN,526.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,30000153,3,2174-09-29 15:00:00,2174-09-29 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,0,0,0,0,0,0,0
4,30000153,4,2174-09-29 16:00:00,2174-09-29 17:00:00,NaN,430.0,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6046503,39999810,107,2115-12-05 12:00:00,2115-12-05 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,1,0,1
6046504,39999810,108,2115-12-05 13:00:00,2115-12-05 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,1,0,1
6046505,39999810,109,2115-12-05 14:00:00,2115-12-05 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,0,0,0,0,1,0,1
6046506,39999810,110,2115-12-05 15:00:00,2115-12-05 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0,0,0,0,1,0,1


In [23]:
curs.execute("select * from mimiciv_derived.urine_output;")
columns_name = [desc[0] for desc in curs.description]
urine_output = pd.DataFrame(curs.fetchall(), columns=columns_name) 
urine_output 

,stay_id,charttime,urineoutput
0,30000153,2174-09-29 12:12:00,280.0
1,30000153,2174-09-29 14:00:00,45.0
2,30000153,2174-09-29 15:00:00,50.0
3,30000153,2174-09-29 16:00:00,50.0
4,30000153,2174-09-29 17:00:00,45.0
...,...,...,...
3321743,39999810,2115-12-01 06:40:00,125.0
3321744,39999810,2115-12-01 10:00:00,275.0
3321745,39999810,2115-12-02 18:06:00,600.0
3321746,39999810,2115-12-04 06:50:00,350.0


In [24]:
curs.execute("select * from mimiciv_derived.first_day_bg;")
columns_name = [desc[0] for desc in curs.description]
first_day_bg = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_bg 

,subject_id,stay_id,lactate_min,lactate_max,ph_min,ph_max,so2_min,so2_max,po2_min,po2_max,...,chloride_min,chloride_max,calcium_min,calcium_max,glucose_min,glucose_max,potassium_min,potassium_max,sodium_min,sodium_max
0,12466550,30000153,1.3,2.1,7.30,7.31,NaN,NaN,215.0,263.0,...,110.0,112.0,1.10,1.16,158.0,176.0,4.2,4.4,141.0,142.0
1,13180007,30000213,0.9,4.9,7.19,7.44,93.0,93.0,56.0,99.0,...,NaN,NaN,NaN,NaN,171.0,171.0,4.8,4.8,NaN,NaN
2,18421337,30000484,2.0,2.0,7.33,7.33,NaN,NaN,21.0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12207593,30000646,1.6,1.6,7.44,7.44,94.0,94.0,71.0,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12980335,30001148,0.9,1.5,7.32,7.53,98.0,99.0,107.0,411.0,...,105.0,105.0,1.16,1.41,100.0,149.0,3.9,6.3,132.0,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16180713,39999301,1.9,6.0,7.33,7.42,NaN,NaN,68.0,320.0,...,110.0,110.0,1.12,1.12,128.0,362.0,4.3,4.3,136.0,136.0
73177,15498623,39999384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73178,11256534,39999552,1.4,2.2,7.44,7.56,NaN,NaN,137.0,429.0,...,102.0,105.0,1.04,1.21,106.0,151.0,2.7,6.4,136.0,139.0
73179,15403458,39999562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
curs.execute("select * from mimiciv_derived.first_day_bg_art;")
columns_name = [desc[0] for desc in curs.description]
first_day_bg_art = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_bg_art 

,subject_id,stay_id,lactate_min,lactate_max,ph_min,ph_max,so2_min,so2_max,po2_min,po2_max,...,chloride_min,chloride_max,calcium_min,calcium_max,glucose_min,glucose_max,potassium_min,potassium_max,sodium_min,sodium_max
0,12466550,30000153,1.3,2.1,7.30,7.31,NaN,NaN,215.0,263.0,...,110.0,112.0,1.10,1.16,158.0,176.0,4.2,4.4,141.0,142.0
1,13180007,30000213,NaN,NaN,7.37,7.37,93.0,93.0,79.0,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18421337,30000484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12207593,30000646,1.6,1.6,7.44,7.44,94.0,94.0,71.0,71.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12980335,30001148,0.9,1.5,7.32,7.48,98.0,99.0,107.0,411.0,...,105.0,105.0,1.16,1.41,100.0,149.0,3.9,5.0,132.0,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16180713,39999301,1.9,6.0,7.33,7.42,NaN,NaN,68.0,320.0,...,110.0,110.0,1.12,1.12,128.0,362.0,4.3,4.3,136.0,136.0
73177,15498623,39999384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73178,11256534,39999552,1.4,2.2,7.44,7.49,NaN,NaN,137.0,429.0,...,102.0,105.0,1.04,1.21,106.0,151.0,2.7,3.7,136.0,139.0
73179,15403458,39999562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
curs.execute("select * from mimiciv_derived.first_day_gcs;")
columns_name = [desc[0] for desc in curs.description]
first_day_gcs = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_gcs 

,subject_id,stay_id,gcs_min,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,9.0,5.0,1.0,3.0,0.0
1,13180007,30000213,14.0,6.0,4.0,4.0,0.0
2,18421337,30000484,5.0,1.0,1.0,3.0,0.0
3,12207593,30000646,15.0,6.0,5.0,4.0,0.0
4,12980335,30001148,3.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...
73176,17533909,32760804,NaN,NaN,NaN,NaN,NaN
73177,10933807,34268613,NaN,NaN,NaN,NaN,NaN
73178,10791103,37377866,NaN,NaN,NaN,NaN,NaN
73179,13351655,31230214,NaN,NaN,NaN,NaN,NaN


In [27]:
curs.execute("select * from mimiciv_derived.first_day_height;")
columns_name = [desc[0] for desc in curs.description]
first_day_height = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_height 

,subject_id,stay_id,height
0,12578987,32563527,173.00
1,14859979,36134276,163.00
2,11470097,36250432,157.00
3,15312343,33258084,157.00
4,13011410,37282334,163.00
...,...,...,...
73176,11539026,30183170,None
73177,18898128,38736701,None
73178,11390883,39793803,None
73179,15288589,38988285,None


In [22]:
curs.execute("select * from mimiciv_derived.first_day_lab;")
columns_name = [desc[0] for desc in curs.description]
first_day_lab = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_lab 

,subject_id,stay_id,hematocrit_min,hematocrit_max,hemoglobin_min,hemoglobin_max,platelets_min,platelets_max,wbc_min,wbc_max,...,bilirubin_indirect_min,bilirubin_indirect_max,ck_cpk_min,ck_cpk_max,ck_mb_min,ck_mb_max,ggt_min,ggt_max,ld_ldh_min,ld_ldh_max
0,13180007,30000213,23.9,27.6,7.4,8.5,219.0,243.0,5.7,7.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,300.0
1,16165135,30003125,38.0,38.0,13.0,13.0,150.0,150.0,9.9,9.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19462678,30003202,24.5,28.3,8.5,9.9,291.0,332.0,17.8,22.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11101925,30004018,25.9,27.1,8.5,9.3,613.0,640.0,29.4,39.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17220323,30004242,31.5,32.6,10.5,11.0,280.0,289.0,7.9,9.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,16362502,39982460,25.3,32.0,7.5,9.9,147.0,192.0,8.5,11.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73177,15119590,39982700,23.0,24.3,6.9,7.2,247.0,264.0,13.0,14.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73178,15530538,39986775,22.3,34.4,7.2,8.3,122.0,144.0,8.0,11.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73179,15368898,39990887,32.2,36.2,10.9,12.9,131.0,169.0,8.8,13.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
curs.execute("select * from mimiciv_derived.first_day_rrt;")
columns_name = [desc[0] for desc in curs.description]
first_day_rrt = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_rrt 

,subject_id,stay_id,dialysis_present,dialysis_active,dialysis_type
0,12466550,30000153,NaN,NaN,None
1,13180007,30000213,NaN,NaN,None
2,18421337,30000484,NaN,NaN,None
3,12207593,30000646,NaN,NaN,None
4,12980335,30001148,NaN,NaN,None
...,...,...,...,...,...
73176,16180713,39999301,NaN,NaN,None
73177,15498623,39999384,NaN,NaN,None
73178,11256534,39999552,NaN,NaN,None
73179,15403458,39999562,NaN,NaN,None


In [29]:
curs.execute("select * from mimiciv_derived.first_day_urine_output;")
columns_name = [desc[0] for desc in curs.description]
first_day_urine_output = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_urine_output 

,subject_id,stay_id,urineoutput
0,15726063,36415879,4580.0
1,10771068,36006309,176.0
2,19094009,30506515,1150.0
3,13011410,37282334,1500.0
4,14304873,35621403,2390.0
...,...,...,...
73176,11749824,33294180,NaN
73177,11816652,38777171,NaN
73178,12365988,39393758,2020.0
73179,15156617,38979073,4250.0


In [31]:
curs.execute("select * from mimiciv_derived.first_day_vitalsign;")
columns_name = [desc[0] for desc in curs.description]
first_day_urine_vitalsign = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_urine_vitalsign 

,subject_id,stay_id,heart_rate_min,heart_rate_max,heart_rate_mean,sbp_min,sbp_max,sbp_mean,dbp_min,dbp_max,...,resp_rate_mean,temperature_min,temperature_max,temperature_mean,spo2_min,spo2_max,spo2_mean,glucose_min,glucose_max,glucose_mean
0,10546885,35947626,75.0,106.0,88.470588,75.0,126.0,95.645833,33.0,68.0,...,19.875000,None,None,None,94.0,100.0,98.365854,84.0,186.0,153.095238
1,10321933,39606087,63.0,86.0,72.523810,101.0,145.0,123.478261,44.0,94.0,...,16.000000,36.61,36.83,36.7660000000000000,93.0,100.0,96.250000,153.0,153.0,153.000000
2,10371497,30409691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,None,None,NaN,NaN,NaN,121.0,167.0,145.750000
3,10080695,32280626,103.0,120.0,111.000000,80.0,146.0,102.888889,43.0,125.0,...,24.307692,36.22,36.67,36.3966666666666667,70.0,100.0,90.954545,92.0,118.0,105.000000
4,10437117,38622644,67.0,109.0,81.666667,128.0,169.0,144.833333,73.0,109.0,...,14.611111,36.39,37.22,36.7680000000000000,95.0,98.0,96.500000,90.0,98.0,94.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,17718683,32586934,68.0,93.0,79.722222,137.0,172.0,153.437500,74.0,98.0,...,20.055556,36.39,37.11,36.6950000000000000,95.0,100.0,97.611111,175.0,273.0,211.666667
73177,11539026,30183170,62.0,90.0,73.230769,79.0,153.0,111.120000,39.0,81.0,...,14.153846,36.72,37.11,36.8783333333333333,91.0,100.0,95.730769,125.0,125.0,125.000000
73178,11374486,35647824,49.0,162.0,78.384615,73.0,150.0,108.230769,26.0,75.0,...,18.450000,36.89,37.83,37.3600000000000000,96.0,100.0,98.974359,85.0,440.0,232.074074
73179,14561266,33344655,68.0,95.0,78.185185,71.0,137.0,104.461538,44.0,81.0,...,17.366667,35.10,35.10,35.1000000000000000,93.0,100.0,98.291667,62.0,225.0,120.285714


In [33]:
curs.execute("select * from mimiciv_derived.first_day_weight;")
columns_name = [desc[0] for desc in curs.description]
first_day_urine_weight = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_urine_weight 

,subject_id,stay_id,weight_admit,weight,weight_min,weight_max
0,12262521,31787291,94.0,99.500000,94.0,105.0
1,19374979,30582233,NaN,91.000000,91.0,91.0
2,11037710,33513979,86.0,86.000000,86.0,86.0
3,15940139,31716752,70.7,70.700000,70.7,70.7
4,17379189,31730379,72.4,72.400000,72.4,72.4
...,...,...,...,...,...,...
73176,18190003,35198152,88.0,88.000000,88.0,88.0
73177,18296202,34000424,95.6,95.600000,95.6,95.6
73178,18898128,38736701,107.0,107.000000,107.0,107.0
73179,11390883,39793803,NaN,90.500000,90.5,90.5


In [34]:
curs.execute("select * from mimiciv_derived.first_day_sofa;")
columns_name = [desc[0] for desc in curs.description]
first_day_sofa = pd.DataFrame(curs.fetchall(), columns=columns_name) 
first_day_sofa 

,subject_id,hadm_id,stay_id,sofa,respiration,coagulation,liver,cardiovascular,cns,renal
0,10546885,25213587,35947626,12,4.0,2.0,0.0,4.0,0.0,2.0
1,10321933,21721466,39606087,1,NaN,0.0,NaN,1.0,0.0,0.0
2,10371497,22500377,30409691,2,0.0,1.0,0.0,NaN,NaN,1.0
3,10080695,23273240,32280626,7,NaN,0.0,0.0,4.0,1.0,2.0
4,10437117,23373117,38622644,1,NaN,0.0,NaN,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
73176,17718683,20228289,32586934,0,NaN,0.0,0.0,0.0,0.0,0.0
73177,11539026,20292752,30183170,2,NaN,0.0,0.0,1.0,1.0,0.0
73178,11374486,23112869,35647824,11,4.0,0.0,1.0,4.0,0.0,2.0
73179,14561266,20806664,33344655,12,3.0,1.0,0.0,4.0,0.0,4.0


In [ ]:
tunnel.close() #터널 더이상 사용하지 않을때 호출해줘야 합니다.